# Wess Zumino 

## N=1 in 1+1

The Hamiltonian with a central difference for the derivative is given by 

\begin{align}
    H=\sum_n&\left[ \frac{p_n^2}{2a} + \frac{a}{2}\left(\frac{\phi_{n+1}-\phi_{n-1}}{2a}\right)^2+\frac{a}{2}V(\phi_n)^2
            +aV(\phi_n)\frac{\phi_{n+1}-\phi_{n-1}}{2a} \right. \nonumber \\
            &\left.+(-1)^nV'(\phi_n)\left(\chi_n^{\dagger}\chi_n-\frac{1}{2}\right)
            +\frac{1}{2a}\left(\chi_n^{\dagger}\chi_{n+1}+\chi_{n+1}^{\dagger}\chi_n\right) \right],
\end{align}

In [1]:
import sympy as sp
from ham_to_matrix import *
from constants import *


In [2]:
bosonNI=sp.expand( pn**2/(2*aLat) + (aLat/2)*((qnP1-qnM1)/(2*aLat))**2 )
bosonI=sp.expand( (aLat/2)*V(qn)**2 + aLat*V(qn)*(qnP1-qnM1)/(2*aLat) )
fermionNI=sp.expand( (xdnP1*xn+xdn*xnP1)/(2*aLat) )
fermionI=sp.expand( sp.diff(V(qn),qn)*(xdn*xn-(1/2)) )

In [3]:
bosonNI+bosonI

a*V(q_{n})**2/2 + V(q_{n})*q_{n+1}/2 - V(q_{n})*q_{n-1}/2 + p_{n}**2/(2*a) - q_{n+1}*q_{n-1}/(8*a) + q_{n+1}**2/(8*a) - q_{n-1}*q_{n+1}/(8*a) + q_{n-1}**2/(8*a)

In [4]:
fermionNI + fermionI

-0.5*Derivative(V(q_{n}), q_{n}) + Derivative(V(q_{n}), q_{n})*\chi^{\dagger}_{n}*\chi_{n} + \chi^{\dagger}_{n+1}*\chi_{n}/(2*a) + \chi^{\dagger}_{n}*\chi_{n+1}/(2*a)

In [5]:
N=2
aVal=1

# depends on finite-difference method
qs=[SiteSymbol('q',str(i)) for i in range(-1,N+1,1)]
# maybe make qs a normal site list
# and make an extra boundaryQs list for q[-1], q[N]
# counting would be normal computer science way for rest of code.

ps=[SiteSymbol('p',str(i)) for i in range(-1,N+1,1)] # don't really need extras
aops=[SiteSymbol('a',str(i)) for i in range(-1,N+1,1)]
adags=[SiteSymbol('a^{\dagger}',str(i)) for i in range(-1,N+1,1)]
xs=[SiteSymbol('\chi',str(i)) for i in range(-1,N+1,1)]
xdags=[SiteSymbol('\chi^{\dagger}',str(i)) for i in range(-1,N+1,1)]

# note this is exactly hardcoded for this finite difference method.
bcType = 'periodic'
boundaryConditions = {}
if bcType == 'periodic':
    boundaryConditions = {qs[0]: qs[N], qs[N+1]: qs[1],
                          xs[0]: -xs[N], xs[N+1]: -xs[1], 
                          xdags[0]: -xdags[N],  xdags[N+1]: -xdags[1]
                         }
    
elif bcType == 'dirichlet':
    boundaryConditions = { qs[0]: 0, qs[N+1]: 0,
                          xs[0]: 0, xs[N+1]: 0,
                          xdags[0]: 0, xdags[N+1]:0
                         }

In [6]:
totHam=bosonNI+bosonI+fermionNI+fermionI
totHam

a*V(q_{n})**2/2 + V(q_{n})*q_{n+1}/2 - V(q_{n})*q_{n-1}/2 - 0.5*Derivative(V(q_{n}), q_{n}) + Derivative(V(q_{n}), q_{n})*\chi^{\dagger}_{n}*\chi_{n} + \chi^{\dagger}_{n+1}*\chi_{n}/(2*a) + \chi^{\dagger}_{n}*\chi_{n+1}/(2*a) + p_{n}**2/(2*a) - q_{n+1}*q_{n-1}/(8*a) + q_{n+1}**2/(8*a) - q_{n-1}*q_{n+1}/(8*a) + q_{n-1}**2/(8*a)

In [7]:
ham=0

lam=1

for i in range(1,N+1):
    ham+=(bosonNI+bosonI+fermionNI).subs({
        pn: ps[i],
        qn: qs[i], qnP1: qs[i+1], qnM1: qs[i-1],
        xn: xs[i], xnP1: xs[i+1],
        xdn: xdags[i], xdnP1: xdags[i+1]
    }).subs(boundaryConditions)
    
# ham.subs(boundaryConditions).doit() # this doesn't work?


def potential(n):
    # m*q with m=1
    return lam*qs[n]*qs[n]*qs[n]

potentialSubs={}
for n in range(1,N+1):
    potentialSubs[V(qs[n])]=potential(n)

ham=sp.simplify(ham.subs(potentialSubs).subs(aLat,aVal))

for i in range(1,N+1):
    if i%2==0:
        ham+=sp.simplify(fermionI.subs({qn: qs[i], xn: xs[i], xdn: xdags[i]}).subs(potentialSubs))
    else:
        ham-=sp.simplify(fermionI.subs({qn: qs[i], xn: xs[i], xdn: xdags[i]}).subs(potentialSubs))
ham=sp.simplify(ham.subs(aLat,aVal))
ham

(p_{0}**2 + p_{1}**2 + 3.0*q_{0}**2 - 6*q_{0}**2*\chi^{\dagger}_{0}*\chi_{0} + q_{0}**6 - 3.0*q_{1}**2 + 6*q_{1}**2*\chi^{\dagger}_{1}*\chi_{1} + q_{1}**6)/2

In [8]:
#ham=0
#totHam=bosonNI+bosonI+fermionNI+fermionI
#for i in range(0,N):
#    ham+=totHam.subs(n,i).subs(boundaryConditions)
# ham.subs(boundaryConditions).doit() # this doesn't work?

#def potential(n):
#    return q[n]

#for n in range(0,N):
#    ham=ham.subs(V(q[n]),potential(n))
#ham=ham.subs(aLat,aVal).simplify().expand()
#ham

In [9]:
m=1

HOdofSubs = {}
#offset because of BC
for i in range(1,N+1):
    HOdofSubs[qs[i]] = 0.5*sp.sqrt(2/m)*(aops[i] + adags[i])
    HOdofSubs[ps[i]] = complex(0,1)*sp.sqrt(2*m)*(adags[i] - aops[i])/2 

hoHam=sp.expand(ham.subs(HOdofSubs))
hoHam=sp.nsimplify(hoHam,tolerance=1e-8)
hoHam

a^{\dagger}_{0}*a_{0} - 3*a^{\dagger}_{0}*a_{0}*\chi^{\dagger}_{0}*\chi_{0}/2 + a^{\dagger}_{0}*a_{0}*a^{\dagger}_{0}*a_{0}*a^{\dagger}_{0}*a_{0}/16 + a^{\dagger}_{0}*a_{0}*a^{\dagger}_{0}*a_{0}*a^{\dagger}_{0}**2/16 + a^{\dagger}_{0}*a_{0}*a^{\dagger}_{0}*a_{0}**2*a^{\dagger}_{0}/16 + a^{\dagger}_{0}*a_{0}*a^{\dagger}_{0}*a_{0}**3/16 + a^{\dagger}_{0}*a_{0}*a^{\dagger}_{0}**2*a_{0}*a^{\dagger}_{0}/16 + a^{\dagger}_{0}*a_{0}*a^{\dagger}_{0}**2*a_{0}**2/16 + a^{\dagger}_{0}*a_{0}*a^{\dagger}_{0}**3*a_{0}/16 + a^{\dagger}_{0}*a_{0}*a^{\dagger}_{0}**4/16 + a^{\dagger}_{0}*a_{0}**2*a^{\dagger}_{0}*a_{0}*a^{\dagger}_{0}/16 + a^{\dagger}_{0}*a_{0}**2*a^{\dagger}_{0}*a_{0}**2/16 + a^{\dagger}_{0}*a_{0}**2*a^{\dagger}_{0}**2*a_{0}/16 + a^{\dagger}_{0}*a_{0}**2*a^{\dagger}_{0}**3/16 + a^{\dagger}_{0}*a_{0}**3*a^{\dagger}_{0}*a_{0}/16 + a^{\dagger}_{0}*a_{0}**3*a^{\dagger}_{0}**2/16 + a^{\dagger}_{0}*a_{0}**4*a^{\dagger}_{0}/16 + a^{\dagger}_{0}*a_{0}**5/16 + a^{\dagger}_{0}**2/2 - 3*a^{\dagger}

In [10]:
cutoff=4
hamMat=convert_to_matrix(hoHam,cutoff,N,aops,adags,xs,xdags)

In [11]:
ens=np.sort(np.linalg.eig(hamMat.astype(np.complex64))[0])
ens=ens.round(8)

print(np.isreal(ens).all())

ens

True


array([ 0.19003424-0.j,  1.0407456 -0.j,  1.0407456 +0.j,  1.0549341 -0.j,
        1.0549341 -0.j,  1.8806992 -0.j,  1.8806992 +0.j,  1.8914568 +0.j,
        1.9056455 -0.j,  1.9056455 +0.j,  1.9198343 +0.j,  2.731411  +0.j,
        2.731411  +0.j,  2.7455993 -0.j,  2.7455993 +0.j,  3.5713644 +0.j,
        6.2601    -0.j,  6.2601    +0.j,  7.1108117 -0.j,  7.1108117 +0.j,
        7.1249995 +0.j,  7.1249995 +0.j,  7.1249995 +0.j,  7.1250005 +0.j,
        7.9507656 +0.j,  7.9507656 +0.j,  7.9757113 -0.j,  7.9757113 +0.j,
        7.9899    +0.j,  7.9899    +0.j,  8.815665  +0.j,  8.815665  +0.j,
       12.330166  -0.j, 13.1950655 -0.j, 13.1950655 -0.j, 14.059967  -0.j,
       14.434335  -0.j, 14.434335  +0.j, 15.274288  -0.j, 15.274288  +0.j,
       15.285047  -0.j, 15.285047  -0.j, 15.299236  +0.j, 15.299236  +0.j,
       16.125     +0.j, 16.125     +0.j, 16.125     +0.j, 16.125     +0.j,
       16.139189  -0.j, 16.139189  +0.j, 16.964954  -0.j, 16.964954  -0.j,
       20.5044    -0.j, 2

In [14]:
((10**2)*(2**2))**2

160000

In [15]:
for cutoff in [2,3,4,5,6,7,8]:
    hamMat=convert_to_matrix(hoHam,cutoff,N,aops,adags,xs,xdags)
    ens=np.sort(np.linalg.eig(hamMat.astype(np.complex64))[0])
    ens=ens.round(8)
    
    print("Lambda = ",cutoff)
    print(ens[0:10])
    print()

Lambda =  2
[-0.875+0.j -0.875+0.j -0.875+0.j -0.875+0.j  0.625+0.j  0.625+0.j
  0.625+0.j  0.625+0.j  0.625+0.j  0.625+0.j]

Lambda =  3
[-1.0515145 +0.j -0.33825737+0.j -0.33825737+0.j  0.12442722+0.j
  0.12442722+0.j  0.18749988+0.j  0.18749988+0.j  0.37499976+0.j
  0.83768433+0.j  0.83768433+0.j]

Lambda =  4
[0.19003424-0.j 1.0407456 -0.j 1.0407456 +0.j 1.0549341 -0.j
 1.0549341 -0.j 1.8806992 -0.j 1.8806992 +0.j 1.8914568 +0.j
 1.9056455 -0.j 1.9056455 +0.j]

Lambda =  5
[-0.86277425+0.j -0.03467345+0.j -0.03467345+0.j  0.50928307-0.j
  0.50928307+0.j  0.793428  +0.j  1.2903428 +0.j  1.2903428 +0.j
  1.3373839 +0.j  1.3373839 +0.j]

Lambda =  6
[0.15034774+0.j 1.0009421 -0.j 1.0009421 -0.j 1.0501926 -0.j
 1.0501926 +0.j 1.8515372 -0.j 1.900787  -0.j 1.900787  +0.j
 1.9500375 +0.j 2.421206  -0.j]



KeyboardInterrupt: 

In [17]:
for cutoff in [7,8,9,10]:
    hamMat=convert_to_matrix(hoHam,cutoff,N,aops,adags,xs,xdags)
    ens=np.sort(np.linalg.eig(hamMat.astype(np.complex64))[0])
    ens=ens.round(8)
    
    print("Lambda = ",cutoff)
    print(ens[0:10])
    print()

Lambda =  7
[-0.19459018+0.j  0.85713005+0.j  0.85713005+0.j  0.8792741 +0.j
  0.8792741 +0.j  1.9088504 +0.j  1.9309944 +0.j  1.9309944 +0.j
  1.9531388 +0.j  2.868149  +0.j]

Lambda =  8
[-0.13005099-0.j  0.4779801 +0.j  0.4779801 +0.j  0.9130962 -0.j
  0.9130962 +0.j  1.086012  -0.j  1.5211273 -0.j  1.5211273 +0.j
  1.9562432 -0.j  2.4092731 -0.j]

Lambda =  9
[0.0647111+0.j 0.9921449+0.j 0.9921449+0.j 1.1588206+0.j 1.1588206+0.j
 1.9195787+0.j 2.0862544+0.j 2.0862544+0.j 2.2529309+0.j 3.1660137-0.j]

Lambda =  10
[-0.08308198+0.j  0.70125854+0.j  0.70125854+0.j  0.92909414-0.j
  0.92909414+0.j  1.4855996 +0.j  1.7134349 -0.j  1.7134349 +0.j
  1.9412704 +0.j  2.962519  -0.j]

